In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from utils import show_recipe

C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_path = "model/finetuned_recipenlg"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model and tokenizer loaded.")


prompt = (
    "<RECIPE_START>"
    "<INPUT_START>milk<NEXT_INPUT>cream<NEXT_INPUT>strawberry<NEXT_INPUT>melon<NEXT_INPUT>honey<INPUT_END>"
)

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=400,
    return_attention_mask=True,
)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=400,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True,
    eos_token_id=tokenizer.convert_tokens_to_ids("<RECIPE_END>"),
)

raw_recipe = tokenizer.decode(outputs[0])
show_recipe(raw_recipe)

Setting `pad_token_id` to `eos_token_id`:50266 for open-end generation.


Model and tokenizer loaded.
## Recipe Title: Melon and Strawberry Ice Cream

Inputs (base ingredients):
   - milk
   - cream
   - strawberry
   - melon
   - honey

Ingredients:
   - 2 cups milk
   - 1 cup half-and-half cream
   - 12 cup strawberry, hulled and quartered
   - 8 ounces cantaloupe, peeled, seeded, and cut into 1/2-inch cubes
   - 2 tablespoons honey

Instructions:
   1. In a medium saucepan, combine the milk and cream.
   2. Bring to a boil over medium-high heat, then reduce the heat to low and simmer, stirring occasionally, until the mixture is thickened, about 5 minutes (do not let it boil).
   3. Stir in the strawberries and melon; cover and refrigerate until chilled, at least 2 hours.
